In [2]:
!pip install pymongo pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 KB 966.3 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.


In [23]:
# 1. Menghubungkan ke Database MongoDB
# Tugas 1: Pastikan MongoDB telah terpasang dan berjalan. Hubungkan ke MongoDB lokal.

from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [24]:
# 2. Operasi CRUD Dasar
# Tugas 2: Insert, Update, dan Delete data pada koleksi employees.
import random

# Daftar departemen
departments = ['Finance', 'HR', 'IT', 'Marketing', 'Sales', 'Operations']

# Hapus data lama agar tidak duplikat
collection.delete_many({})

# Insert 100 data karyawan
for i in range(1, 101):
    employee_data = {
        'employee_id': i,
        'name': f'Employee_{i}',
        'department': random.choice(departments),
        'age': random.randint(21, 55),
        'salary': random.randint(3000, 12000)
    }
    collection.insert_one(employee_data)

print("✅ 100 data karyawan berhasil ditambahkan ke koleksi 'employees'")

# Menampilkan 10 data pertama untuk memastikan data sudah masuk
print("\n📋 Data yang berhasil ditambahkan (10 data pertama):")
for emp in collection.find().limit(10):
    print(emp)

# Update Data
result = collection.update_many(
    {'department': 'IT'},  # kondisi
    {'$mul': {'salary': 1.1}}  # operasi: kalikan salary 1.1 (naik 10%)
)
print(f"\n✅ {result.modified_count} data karyawan departemen IT berhasil diperbarui.")

# Delete Data
result = collection.delete_many({'age': {'$gt': 50}})
print(f"🗑️ {result.deleted_count} data karyawan dengan usia > 50 tahun berhasil dihapus.")

print("Jumlah data tersisa:", collection.count_documents({}))

✅ 100 data karyawan berhasil ditambahkan ke koleksi 'employees'

📋 Data yang berhasil ditambahkan (10 data pertama):
{'_id': ObjectId('6904adecf1775fffd21890c2'), 'employee_id': 1, 'name': 'Employee_1', 'department': 'Finance', 'age': 29, 'salary': 11104}
{'_id': ObjectId('6904adecf1775fffd21890c3'), 'employee_id': 2, 'name': 'Employee_2', 'department': 'IT', 'age': 28, 'salary': 9667}
{'_id': ObjectId('6904adecf1775fffd21890c4'), 'employee_id': 3, 'name': 'Employee_3', 'department': 'Operations', 'age': 26, 'salary': 11141}
{'_id': ObjectId('6904adecf1775fffd21890c5'), 'employee_id': 4, 'name': 'Employee_4', 'department': 'Operations', 'age': 27, 'salary': 10639}
{'_id': ObjectId('6904adecf1775fffd21890c6'), 'employee_id': 5, 'name': 'Employee_5', 'department': 'Operations', 'age': 54, 'salary': 10289}
{'_id': ObjectId('6904adecf1775fffd21890c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 39, 'salary': 6851}
{'_id': ObjectId('6904adecf1775fffd21890c8'), 'em

In [25]:
# Contoh Query Data
print("📋Data Karyawan dari Koleksi 'employees'📋\n")
for employee in collection.find():
    print(employee)

📋Data Karyawan dari Koleksi 'employees'📋

{'_id': ObjectId('6904adecf1775fffd21890c2'), 'employee_id': 1, 'name': 'Employee_1', 'department': 'Finance', 'age': 29, 'salary': 11104}
{'_id': ObjectId('6904adecf1775fffd21890c3'), 'employee_id': 2, 'name': 'Employee_2', 'department': 'IT', 'age': 28, 'salary': 10633.7}
{'_id': ObjectId('6904adecf1775fffd21890c4'), 'employee_id': 3, 'name': 'Employee_3', 'department': 'Operations', 'age': 26, 'salary': 11141}
{'_id': ObjectId('6904adecf1775fffd21890c5'), 'employee_id': 4, 'name': 'Employee_4', 'department': 'Operations', 'age': 27, 'salary': 10639}
{'_id': ObjectId('6904adecf1775fffd21890c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 39, 'salary': 6851}
{'_id': ObjectId('6904adecf1775fffd21890c8'), 'employee_id': 7, 'name': 'Employee_7', 'department': 'HR', 'age': 37, 'salary': 5854}
{'_id': ObjectId('6904adecf1775fffd21890c9'), 'employee_id': 8, 'name': 'Employee_8', 'department': 'Marketing', 'age': 48, 'salar

In [26]:
# 3. Query Lebih Kompleks Menggunakan Aggregation
# Tugas 3: Terapkan aggregation untuk menghitung rata-rata gaji per departemen.

# Query Aggregation untuk mencari rata-rata gaji per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]

print("📊 Rata-rata Gaji per Departemen:\n")
for result in collection.aggregate(pipeline):
    print(result)

📊 Rata-rata Gaji per Departemen:

{'_id': 'IT', 'average_salary': 8506.09375}
{'_id': 'Operations', 'average_salary': 8702.333333333334}
{'_id': 'HR', 'average_salary': 7007.2}
{'_id': 'Marketing', 'average_salary': 7843.333333333333}
{'_id': 'Sales', 'average_salary': 6551.083333333333}
{'_id': 'Finance', 'average_salary': 7394.695652173913}


In [27]:
# 5. TUGAS
# TUGAS 1: Aggregation Pipeline untuk mencari 5 karyawan dengan gaji tertinggi di setiap departemen

pipeline_top5 = [
    # Urutkan semua data berdasarkan 'department' (ascending) dan 'salary' (descending)
    {'$sort': {'department': 1, 'salary': -1}},

    # Kelompokkan berdasarkan departemen
    {'$group': {
        '_id': '$department',
        'top_employees': {
            '$push': {  # Simpan semua data karyawan dalam daftar (array)
                'name': '$name',
                'salary': '$salary',
                'age': '$age',
                'employee_id': '$employee_id'
            }
        }
    }},

    # Ambil hanya 5 data karyawan teratas (gaji tertinggi) dalam setiap departemen
    {'$project': {
        '_id': 1,
        'top_5_employees': {'$slice': ['$top_employees', 5]}  # Ambil 5 elemen pertama dari array
    }}
]

results = collection.aggregate(pipeline_top5)

# Tampilkan hasil 
print("🏆 5 Karyawan dengan Gaji Tertinggi per Departemen\n")
for dept in results:
    print(f"Departemen: {dept['_id']}")
    print("Top 5 Karyawan:")
    for emp in dept['top_5_employees']:
        print(f"{emp['name']} | Gaji: ${emp['salary']} | Usia: {emp['age']} | ID: {emp['employee_id']}")
    print()

🏆 5 Karyawan dengan Gaji Tertinggi per Departemen

Departemen: HR
Top 5 Karyawan:
Employee_83 | Gaji: $9589 | Usia: 37 | ID: 83
Employee_93 | Gaji: $9164 | Usia: 33 | ID: 93
Employee_21 | Gaji: $8410 | Usia: 40 | ID: 21
Employee_44 | Gaji: $7790 | Usia: 48 | ID: 44
Employee_69 | Gaji: $7029 | Usia: 41 | ID: 69

Departemen: IT
Top 5 Karyawan:
Employee_97 | Gaji: $13048.2 | Usia: 50 | ID: 97
Employee_64 | Gaji: $12833.7 | Usia: 22 | ID: 64
Employee_82 | Gaji: $11997.7 | Usia: 30 | ID: 82
Employee_30 | Gaji: $11117.7 | Usia: 39 | ID: 30
Employee_2 | Gaji: $10633.7 | Usia: 28 | ID: 2

Departemen: Marketing
Top 5 Karyawan:
Employee_66 | Gaji: $11966 | Usia: 31 | ID: 66
Employee_49 | Gaji: $11197 | Usia: 50 | ID: 49
Employee_68 | Gaji: $10337 | Usia: 43 | ID: 68
Employee_70 | Gaji: $9979 | Usia: 37 | ID: 70
Employee_52 | Gaji: $9304 | Usia: 44 | ID: 52

Departemen: Operations
Top 5 Karyawan:
Employee_46 | Gaji: $11996 | Usia: 30 | ID: 46
Employee_62 | Gaji: $11421 | Usia: 50 | ID: 62
Employe

In [28]:
# Tugas 2: Menghapus karyawan yang berusia di bawah 25 tahun

# Menampilkan jumlah data sebelum penghapusan
print("Jumlah data sebelum dihapus:", collection.count_documents({}))

# Menghapus semua karyawan yang berusia di bawah 30 tahun
delete_result = collection.delete_many({'age': {'$lt': 30}})

# Menampilkan jumlah data yang dihapus
print(f"Jumlah data karyawan yang berusia di bawah 30 tahun yang dihapus: {delete_result.deleted_count}")

# Menampilkan jumlah data setelah penghapusan
print("Jumlah data setelah dihapus:", collection.count_documents({}))

# Menampilkan semua data karyawan yang tersisa untuk verifikasi
print("\nSemua Data Karyawan yang Tersisa Setelah Penghapusan:")
for employee in collection.find():
    print(employee)

Jumlah data sebelum dihapus: 82
Jumlah data karyawan yang berusia di bawah 30 tahun yang dihapus: 24
Jumlah data setelah dihapus: 58

Semua Data Karyawan yang Tersisa Setelah Penghapusan:
{'_id': ObjectId('6904adecf1775fffd21890c7'), 'employee_id': 6, 'name': 'Employee_6', 'department': 'Sales', 'age': 39, 'salary': 6851}
{'_id': ObjectId('6904adecf1775fffd21890c8'), 'employee_id': 7, 'name': 'Employee_7', 'department': 'HR', 'age': 37, 'salary': 5854}
{'_id': ObjectId('6904adecf1775fffd21890c9'), 'employee_id': 8, 'name': 'Employee_8', 'department': 'Marketing', 'age': 48, 'salary': 3402}
{'_id': ObjectId('6904adecf1775fffd21890ca'), 'employee_id': 9, 'name': 'Employee_9', 'department': 'Sales', 'age': 38, 'salary': 6714}
{'_id': ObjectId('6904adecf1775fffd21890cb'), 'employee_id': 10, 'name': 'Employee_10', 'department': 'Marketing', 'age': 30, 'salary': 4827}
{'_id': ObjectId('6904adecf1775fffd21890cc'), 'employee_id': 11, 'name': 'Employee_11', 'department': 'Finance', 'age': 40, '

In [29]:
# Tugas 3
# Laporan ringkas: total gaji karyawan di setiap departemen dan rata-rata umur karyawan.

# Aggregation pipeline untuk menghitung total gaji dan rata-rata umur per departemen
pipeline = [
    {
        '$group': {
            '_id': '$department',
            'total_salary': {'$sum': '$salary'},
            'average_age': {'$avg': '$age'}
        }
    },
    {
        '$sort': {'_id': 1}  # Mengurutkan hasil berdasarkan nama departemen
    }
]

# Menampilkan hasil laporan
print("📋 Laporan Ringkas: Total Gaji dan Rata-rata Umur per Departemen\n")
for result in collection.aggregate(pipeline):
    print(f"Departemen: {result['_id']}")
    print(f"  Total Gaji     : {result['total_salary']}")
    print(f"  Rata-rata Umur : {result['average_age']:.2f} tahun\n")

📋 Laporan Ringkas: Total Gaji dan Rata-rata Umur per Departemen

Departemen: Finance
  Total Gaji     : 118681
  Rata-rata Umur : 41.41 tahun

Departemen: HR
  Total Gaji     : 54355
  Rata-rata Umur : 40.71 tahun

Departemen: IT
  Total Gaji     : 91703.70000000001
  Rata-rata Umur : 39.64 tahun

Departemen: Marketing
  Total Gaji     : 67400
  Rata-rata Umur : 41.38 tahun

Departemen: Operations
  Total Gaji     : 62835
  Rata-rata Umur : 39.86 tahun

Departemen: Sales
  Total Gaji     : 57945
  Rata-rata Umur : 37.00 tahun



In [41]:
# Tugas Tambahan

# Latihan 4.1: Buatlah koleksi baru products dan masukkan 100 data produk.
# Membuat koleksi baru 'products'
products_collection = db["products"]

# Menghapus data lama agar tidak duplikat
products_collection.delete_many({})

# Daftar kategori dan nama produk
categories = ["Elektronik", "Pakaian", "Makanan", "Peralatan Rumah", "Aksesoris", "Olahraga", "Kesehatan", "Mainan"]
product_names = [
    "Smartphone", "Laptop", "Kamera", "Headphone", "Keyboard", "Mouse", "Monitor", "TV",
    "Sepatu", "Jaket", "Kaos", "Celana", "Kemeja", "Topi", "Tas", "Jam Tangan",
    "Snack", "Minuman", "Roti", "Susu", "Coklat", "Kopi", "Mie Instan", "Saus",
    "Piring", "Sendok", "Kompor", "Vacuum Cleaner", "Blender", "Kulkas",
    "Cincin", "Kalung", "Gelang", "Anting", "Kacamata",
    "Bola", "Raket", "Treadmill", "Matras", "Sepeda",
    "Masker", "Vitamin", "Obat", "Suplemen", "Timbangan",
    "Puzzle", "Robot", "Boneka", "Lego", "Kartu Game"
]

# Membuat 100 data produk random
product_data = []
for i in range(100):
    product = {
        "product_id": i + 1,
        "name": random.choice(product_names) + f" {random.randint(100,999)}",
        "category": random.choice(categories),
        "price": random.randint(10000, 2000000),
        "stock": random.randint(0, 500),
        "rating": round(random.uniform(1.0, 5.0), 1)
    }
    product_data.append(product)

# Menambahkan data ke koleksi
products_collection.insert_many(product_data)

print("✅ Sebanyak 100 data produk berhasil dimasukkan ke koleksi 'products'!")

# Menampilkan data produk yang dibuat
print("\n📋 Data Produk yang Telah Dimasukkan:")
for product in products_collection.find():
    print(product)

✅ Sebanyak 100 data produk berhasil dimasukkan ke koleksi 'products'!

📋 Data Produk yang Telah Dimasukkan:
{'_id': ObjectId('6904af2bf1775fffd218931a'), 'product_id': 1, 'name': 'Suplemen 954', 'category': 'Aksesoris', 'price': 1806360, 'stock': 109, 'rating': 1.2}
{'_id': ObjectId('6904af2bf1775fffd218931b'), 'product_id': 2, 'name': 'Jaket 551', 'category': 'Aksesoris', 'price': 1337849, 'stock': 401, 'rating': 4.2}
{'_id': ObjectId('6904af2bf1775fffd218931c'), 'product_id': 3, 'name': 'Puzzle 729', 'category': 'Peralatan Rumah', 'price': 305537, 'stock': 499, 'rating': 4.8}
{'_id': ObjectId('6904af2bf1775fffd218931d'), 'product_id': 4, 'name': 'Kamera 520', 'category': 'Kesehatan', 'price': 726507, 'stock': 125, 'rating': 4.7}
{'_id': ObjectId('6904af2bf1775fffd218931e'), 'product_id': 5, 'name': 'Minuman 110', 'category': 'Makanan', 'price': 45744, 'stock': 391, 'rating': 2.6}
{'_id': ObjectId('6904af2bf1775fffd218931f'), 'product_id': 6, 'name': 'Puzzle 989', 'category': 'Makanan

In [42]:
# Latihan 4.2: Lakukan query untuk menemukan produk dengan harga di atas rata-rata.

# Hitung harga rata-rata semua produk
average_price_pipeline = [
    {'$group': {'_id': None, 'average_price': {'$avg': '$price'}}}
]
average_price_result = list(products_collection.aggregate(average_price_pipeline))
average_price = average_price_result[0]['average_price']

print(f"💵 Harga rata-rata semua produk: {average_price:.2f}\n")

# Query produk dengan harga di atas rata-rata
above_average_products = products_collection.find({'price': {'$gt': average_price}})

print("📦 Produk dengan Harga di Atas Rata-rata;")
for product in above_average_products:
    print(f"ID Produk   : {product['product_id']}")
    print(f"Nama Produk : {product['name']}")
    print(f"Kategori    : {product['category']}")
    print(f"Harga       : {product['price']}")
    print(f"Stok        : {product['stock']}")
    print(f"Rating      : {product['rating']}\n")

💵 Harga rata-rata semua produk: 1030167.68

📦 Produk dengan Harga di Atas Rata-rata;
ID Produk   : 1
Nama Produk : Suplemen 954
Kategori    : Aksesoris
Harga       : 1806360
Stok        : 109
Rating      : 1.2

ID Produk   : 2
Nama Produk : Jaket 551
Kategori    : Aksesoris
Harga       : 1337849
Stok        : 401
Rating      : 4.2

ID Produk   : 7
Nama Produk : Celana 723
Kategori    : Olahraga
Harga       : 1637456
Stok        : 392
Rating      : 4.2

ID Produk   : 8
Nama Produk : Mie Instan 797
Kategori    : Mainan
Harga       : 1107545
Stok        : 243
Rating      : 3.6

ID Produk   : 10
Nama Produk : Blender 654
Kategori    : Elektronik
Harga       : 1330466
Stok        : 184
Rating      : 2.3

ID Produk   : 11
Nama Produk : Roti 501
Kategori    : Elektronik
Harga       : 1610108
Stok        : 133
Rating      : 2.3

ID Produk   : 17
Nama Produk : Piring 928
Kategori    : Kesehatan
Harga       : 1620706
Stok        : 189
Rating      : 4.2

ID Produk   : 18
Nama Produk : Jaket 449
K

In [43]:
# Latihan 4.3: Buatlah aggregation pipeline untuk menghitung total produk dalam setiap kategori.
# Aggregation pipeline untuk menghitung total produk per kategori
pipeline = [
    {
        '$group': {
            '_id': '$category',
            'total_products': {'$sum': 1}
        }
    },
    {
        '$sort': {'total_products': -1}  # urutkan dari kategori dengan jumlah produk terbanyak
    }
]

# Menjalankan pipeline dan menampilkan hasil
print("Total Produk per Kategori: \n")
for result in products_collection.aggregate(pipeline):
    print(f"Kategori       : {result['_id']}")
    print(f"Total Produk  : {result['total_products']}\n")

Total Produk per Kategori: 

Kategori       : Aksesoris
Total Produk  : 22

Kategori       : Peralatan Rumah
Total Produk  : 16

Kategori       : Kesehatan
Total Produk  : 15

Kategori       : Makanan
Total Produk  : 12

Kategori       : Mainan
Total Produk  : 11

Kategori       : Elektronik
Total Produk  : 10

Kategori       : Olahraga
Total Produk  : 9

Kategori       : Pakaian
Total Produk  : 5

